## Imports

In [61]:
import requests
from bs4 import BeautifulSoup
import csv

## Getting HTML

In [62]:
URL = 'https://www.pro-football-reference.com/play-index/draft-finder.cgi?request=1&year_min=1970&year_max=2019&draft_slot_min=1&draft_slot_max=500&pick_type=overall&pos%5B%5D=qb&pos%5B%5D=rb&pos%5B%5D=wr&pos%5B%5D=te&pos%5B%5D=e&pos%5B%5D=t&pos%5B%5D=g&pos%5B%5D=c&pos%5B%5D=ol&pos%5B%5D=dt&pos%5B%5D=de&pos%5B%5D=dl&pos%5B%5D=ilb&pos%5B%5D=olb&pos%5B%5D=lb&pos%5B%5D=cb&pos%5B%5D=s&pos%5B%5D=db&pos%5B%5D=k&pos%5B%5D=p&conference=any&show=all&order_by=default'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

## Scrapping

In [63]:
def get_all_rows(soup):
    table = soup.find(id='results')
    tbody = table.find('tbody')
    trs = tbody.find_all('tr')

    return trs

In [73]:
def isEmpty(td):
    return td is None # or len(td) <= 0


def scrapy_row(tr):
    identity = None
    body = None

    td_year = tr.find('td', {'data-stat': 'year_id'})
    td_round = tr.find('td', {'data-stat': 'draft_round'})
    td_pick = tr.find('td', {'data-stat': 'draft_pick'})

    td_name = tr.find('td', {'data-stat': 'player'})
    td_position = tr.find('td', {'data-stat': 'pos'})
    td_age = tr.find('td', {'data-stat': 'draft_age'})
    td_first_team_ap = tr.find('td', {'data-stat': 'all_pros_first_team'})
    td_pro_bowls = tr.find('td', {'data-stat': 'pro_bowls'})
    td_team = tr.find('td', {'data-stat': 'team'})
    td_av = tr.find('td', {'data-stat': 'career_av'})
    if isEmpty(td_year): #or isEmpty(td_round) or isEmpty(td_pick) or isEmpty(td_av):
        return None

    data = {}

    data['year'] = td_year.find('a').text
    data['round'] = td_round.text
    data['pick'] = td_pick.text
    data['player_name'] = td_name.text
    data['position'] = td_position.text
    data['age'] = td_age.text
    data['first_team_ap'] = td_first_team_ap.text
    data['pro_bowls'] = td_pro_bowls.text
    data['team'] = td_team.text

    av = td_av.text
    if av == '':
        av = '0'
    data['av'] = av

    return data

In [74]:
trs = get_all_rows(soup)
players = []
for tr in trs:
     player = scrapy_row(tr)
     if player is None:
          continue

     players.append(player)

In [75]:
with open('drafted_players.csv', 'w', newline='') as csvfile:
    fieldnames = ['year', 'round', 'pick', 'player_name', 'position', 'age', 'first_team_ap',                    'pro_bowls', 'team', 'av']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for player in players:
        writer.writerow(player)